In [ ]:
directory = os.fsencode('./data')
series = []
for file in os.scandir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.json'):
        try:
            with open(filename) as f:
                index = re.search('(?<=-).+(?= )', f.name).group()  # extract date
                data = json.load(f)
                # backwards compatiability for data sets that didn't have residuals calculated
                if 'values' in data:
                    json_series = pd.Series(data['residuals'])
                else:
                    json_series = pd.Series(data)

                series.append(json_series.rename(index))
        except ValueError:
            continue

df = pd.DataFrame(series).sort_index().T
df.index = pd.to_datetime(df.index)
df_monthly = df.resample('M').std() # convert to IVOL
df_monthly.apply(lambda x: x[x >= x.quantile(0.8)], axis=1) # highest IVOL


